In [179]:
import pandas as pd
import os

path = "C:/Users/danna/Documents/GitHub/WBBTournamentPredictions/MachineLearning/TestData"
yr_list = os.listdir(path)
print(yr_list)
gamesLists = []
for yr in yr_list:
    path2 = path + "/" + yr
    gamesLists.append(os.listdir(path2))
team1File = "Team 1 Basic BS"
team2File = "Team 2 Basic BS" 
ext = ".csv"
yrIndex = 0
gameIndex = 0

base_cols = None

games = []
gameID = 0

print("\n\nSTART")

for yr in range( len(yr_list) ):
    for gameFile in range( len(gamesLists[yr]) ):
#         print("\t", gamesLists[yr][gameFile])
        testGamePath = path + '/' + yr_list[yr] + '/' + gamesLists[yr][gameFile] 
        team1DataPath = testGamePath + '/' + team1File + ext
        team2DataPath = testGamePath + '/' + team2File + ext
        try:
            team1 = pd.read_csv(team1DataPath)
            team2 = pd.read_csv(team2DataPath)
        except: 
            print("error with", gamesLists[yr][gameFile], "(", yr_list[yr], ")... skipping.")
            continue
        
        teams = [team1, team2]
        summaries = []
        for team in teams:
            
            team.drop(["Unnamed: 0"], axis=1, inplace=True)
            team.iloc[-1, 0] = team.columns[0]
            team.rename(columns={team.columns[0] : "TEAM"}, inplace=True)    

            team["YEAR"] = yr_list[yrIndex]
            
            totals = pd.concat([team.iloc[-1, :]])
            totals.index = totals.index.str.upper()

            maxes = pd.concat([team.iloc[:-1, :].max()])
            maxes.index = maxes.index.str.upper() + "_MAX"

            summary = pd.concat([totals, maxes])
            summaries.append(summary)
        summary = pd.concat(summaries, axis=1).T
        
        game = pd.concat(summaries, axis=1).T
        game_opp = game.iloc[::-1].reset_index()
        game_opp.columns += "_OPP"

        full_game = pd.concat([game, game_opp], axis=1)
        full_game.drop(["YEAR_MAX", "YEAR_OPP", "YEAR_MAX_OPP", "TEAM_MAX", "TEAM_MAX_OPP"], axis=1, inplace=True)

#         full_game["WIN"] = full_game["PTS"] > full_game["PTS_OPP"]
        full_game["WIN"] = None
        full_game["GAME_ID"] = gameID
        gameID += 1
        games.append(full_game)

        if len(games) % 50 == 0:
            print(f"{len(games)} / {len(yr_list) * len(gamesLists)}")

print("END")

['2018', '2021', '2022']


START
50 / 9
100 / 9
150 / 9
END


In [180]:
#run after last cell finishes big loop 
games_df = pd.concat(games, ignore_index=True)
# games_df[-1]
games_df

TEAM  FG FGA  2P 2PA  3P 3PA  FT FTA ORB  ... AST_MAX_OPP  \
0         American   24  56  15  33   9  23   3   8  11  ...          11   
1             UCLA   26  60  18  45   8  15  11  14  11  ...           6   
2    Arizona State   23  59  19  45   4  14  15  20   3  ...           5   
3            Texas   37  60  34  50   3  10   8  10  10  ...           3   
4          Belmont   21  54  10  26  11  28   5  10  11  ...           5   
..              ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...         ...   
371          Texas   35  55  32  49   3   6   5  10   6  ...           5   
372      Villanova   22  59  16  33   6  26  11  17   9  ...           8   
373  Brigham Young   21  54  15  37   6  17   9  12   7  ...           3   
374      Villanova   19  58  13  37   6  21   5   7   6  ...           3   
375       Michigan   24  57  22  44   2  13  14  21  16  ...           2   

    STL_MAX_OPP BLK_MAX_OPP TOV_MAX_OPP PF_MAX_OPP PTS_MAX_OPP FT/FGA_MAX_OPP  \
0             3           3           2          2          20          0.375   
1             2           1           8          4          22            0.4   
2             1           0           5          3          19            0.5   
3             2           0           2          5          27       0.692308   
4             3           2           3          3          25            1.0   
..          ...         ...         ...        ...         ...            ...   
371           2           1           4          3          18       0.222222   
372           3           0           5          4          21       0.470588   
373           6           2           2          4          25            0.6   
374           5           0           4          2          27            1.0   
375           2           3           3          5          12       0.333333   

    AST/TOV_MAX_OPP   WIN GAME_ID  
0               5.5  None       0  
1               3.0  None       0  
2               2.5  None       1  
3               3.0  None       1  
4               5.0  None       2  
..              ...   ...     ...  
371             2.0  None     185  
372             1.6  None     186  
373             3.0  None     186  
374             2.0  None     187  
375             1.0  None     187  

[376 rows x 78 columns]

In [184]:
games_df.to_csv(path + '/testData.csv')